In [1]:
import os
from clean_data import CleanTabular, CleanImage
import numpy as np
from PIL import Image
from pytorch import FbDataset, CNN, Resnet50, train, accuracy
from torch.utils.data import DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split

In [ ]:
image_path = os.path.join(os.path.dirname(os.getcwd()), 'image_data', 'fb_image')
image_cleaner = CleanImage(image_path)
cleaned_img_folder = image_cleaner.process_images(resized_pixel=(128, 128), mode = 'RGB')

In [2]:
parent_path = os.path.dirname(os.getcwd())
cleaned_img_folder = os.path.join(parent_path, "cleaned_image_data")
cleaned_image_paths = []
for dirpaths, dirnames, filenames in os.walk(cleaned_img_folder):
    for filename in filenames:
        cleaned_image_paths.append(os.path.join(cleaned_img_folder, filename))


In [3]:
tabular_cleaner = CleanTabular(os.path.join(os.path.dirname(os.getcwd()), 'data/Images.csv'))
tabular_cleaner.slice_df(['id', 'product_id'])
image_df = tabular_cleaner.df
image_df['image_data'] = np.NaN
image_df['image_data']=image_df['image_data'].astype('object')

convert_tensor = transforms.ToTensor()

for i in range(len(image_df['id'])):
    img = Image.open(os.path.join(cleaned_img_folder, image_df['id'][i] + '.jpg'))
    tensor = convert_tensor(img)
    numpydata = tensor.numpy()
    image_df.at[i, 'image_data'] = numpydata

In [4]:
tabular_cleaner = CleanTabular(os.path.join(os.path.dirname(os.getcwd()), 'data/Products.csv'))
tabular_cleaner.slice_df(['id', 'category'])
tabular_cleaner.clean_to_general_category('category')
cat_code_mapper = tabular_cleaner.clean_to_category_type('category', ohe=False)
product_df = tabular_cleaner.df
product_df = product_df.rename(columns={'id':'product_id'})

In [5]:
merged_df = image_df.merge(product_df, how='left', on='product_id')
merged_df = merged_df.drop(['product_id'], axis=1)
merged_df = merged_df.drop(['id'], axis=1)
# merged_df.to_csv('merged_df.csv', index = False)

In [7]:
fb_dataset = FbDataset(merged_df)

In [8]:
train_set, discard = torch.utils.data.random_split(fb_dataset, [1000, 11604])
train_set, test_set = torch.utils.data.random_split(train_set, [800, 200])

In [9]:
train_loader = DataLoader(train_set, shuffle = True, batch_size = 16)

In [ ]:
model = CNN(3, 565504)
train(model, train_loader, 0.001, epochs = 5)

In [10]:
model_resnet50 = Resnet50()
accuracy(model_resnet50, train_loader)
# train(model_resnet50, train_loader, 0.03, epochs = 3, model_name='resnet50_200dataaset')
# accuracy(model_resnet50, train_loader)

Using cache found in /Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


<class 'torch.Tensor'>
torch.Size([16, 3, 128, 128])


KeyboardInterrupt: 

In [15]:
new_model = Resnet50()
state_dict = torch.load(os.path.join('model_evaluation', 'resnet50_200dataaset','weights', "2022-10-22 21:42:18.018291.pt"))
new_model.load_state_dict(state_dict)
# accuracy(new_model, train_set)

Using cache found in /Users/serenawong/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


<All keys matched successfully>

In [12]:
image_path = os.path.join(os.path.dirname(os.getcwd()), 'test_image')
image_cleaner = CleanImage(image_path)
cleaned_img_folder = image_cleaner.process_images(resized_pixel=(128, 128), mode = 'RGB', replace = True)
convert_tensor = transforms.ToTensor()
img = Image.open('/Users/serenawong/Desktop/fb_marketplace/ffdbe092-377b-4e90-8437-1faa964e5bf4.jpg')
img_tensor = convert_tensor(img)
img_numpy = img_tensor.numpy()
img_numpy = np.expand_dims(img_numpy, axis=0)
img_torch = torch.from_numpy(img_numpy)

In [17]:
img_torch.shape

torch.Size([1, 3, 128, 128])

In [18]:
outputs = new_model(img_torch)
_, predicted = torch.max(outputs.data, 1)

In [19]:
predicted


tensor([6])